In [1]:
import pandas as pd
# from sunpy.coordinates.transformations import hgs_to_hcc
# from sunpy.coordinates import frames
import sunpy.coordinates
from astropy.coordinates import SkyCoord
from math import isnan, ceil, floor
import numpy as np
import astropy.units as u
import matplotlib.pylab as plt


In [21]:
data = pd.read_csv("distance_data.csv")
# print(data.drop(columns = [
#                           'Unnamed: 0'], inplace = True))
# data.dropna(axis = 0, subset = ["hc_x", "hc_y", "hc_z"], inplace = True)
# data.to_csv("distance_data.csv", index=False)
data.head()
# data.iloc[9999]

,Unnamed: 0,HARPNUM,T_REC,NOAA_AR,LAT_FWT,LON_FWT,AREA_ACR,USFLUX,MEANGAM,MEANGBT,...,M_flare_in_24h,any_flare_in_24h,X_flare_in_48h,M_flare_in_48h,any_flare_in_48h,hc_x,hc_y,hc_z,closest_dist,connected_components_mean_thresh
0,0,14,2010-05-05 03:12:00,11070,NaN,NaN,0.00000,2.040540e+18,82.474,38.177,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,1,14,2010-05-05 04:00:00,11070,20.684654,-1.045460,15.91787,3.406072e+20,56.375,119.260,...,0,0,0,0,0,-11872.018458,287843.714077,633037.462958,20.576242,NaN
2,2,14,2010-05-05 05:00:00,11070,20.507109,-0.913314,25.38142,5.640165e+20,49.561,135.720,...,0,0,0,0,0,-10383.617824,285833.795234,633973.658796,20.416992,1.0
3,3,14,2010-05-05 06:00:00,11070,20.496035,-0.419703,31.01844,6.893843e+20,46.385,156.766,...,0,0,0,0,0,-4772.177056,285667.229696,634115.792476,20.437609,1.0
4,4,14,2010-05-05 07:00:00,11070,20.480949,0.125770,31.89329,6.563502e+20,44.189,170.519,...,0,0,0,0,0,1430.203374,285452.974486,634228.611732,20.351355,1.0


In [36]:
ind = 5
t_rec = data.loc[data.index[ind], "T_REC"]
vals = data.loc[data["T_REC"]==t_rec]
print(vals.loc[:,"any_flare_in_24h"])

5       0
176     0
298     1
432     0
972     0
1270    0
Name: any_flare_in_24h, dtype: int64


In [40]:
def calculate_distance_graph(series):
    # get all indices that match, get skycoord objects
    skycoord_list = []
    count=series.shape[0]
    # Error because this line only returns 1 row.
    for ind in range(count):
        row = series.loc[series.index[ind]]
        skycoord_list.append(SkyCoord(row["LON_FWT"]*u.deg,
                                      row["LAT_FWT"]*u.deg,
                                      obstime = row["T_REC"],
                                      frame = "heliographic_stonyhurst"))

    dist_graph = dict()
    # Initialize dict
    for i in range(count):
        dist_graph[i] = np.zeros(count)

    for i in range(count):
        for j in range(i, series.shape[0]):
            if i==j:
                dist_graph[i][j] = float('inf')
                dist_graph[j][i] = float('inf')
            else:
                # Calculate the Great-Circle distance
                dist = skycoord_list[i].separation(skycoord_list[j])
                dist_graph[i][j] = dist.degree
                dist_graph[j][i] = dist.degree

    # print(dist_graph)
    # print(skycoord_list)
    return dist_graph

def thresholded_distance_graph(dist_graph, threshold):
    # Given a distance graph and a threshold, remove all connections with distance greater than threshold and return
    # a new graph
    thresholded_graph = dict()
    # initialize graph

    for key in dist_graph.keys():
        thresholded_graph[key] = np.zeros(len(dist_graph[key]))
        for ind, val in enumerate(dist_graph[key]):
            if val <= threshold:
                thresholded_graph[key][ind] = val
        thresholded_graph[key][key] = float('inf')
    return thresholded_graph

def get_connected_components(graph):
    # Given a graph in the form of an adjacency list with a dictionary, return a 2d list containing the indices of each
    # component. I.e. if there were 2 components in a 6 node graph, with 3 nodes in eachcomponent, this would return a
    # 2x3 2d array.
    def _visit_val(index):
        visited[index] = True
        for i in range(len(graph[index])):
            if graph[index][i] != 0 and i != index and not visited[i]:
                print("Adding {} to component list {}".format(i, component_list))
                component_list.append(i)
                _visit_val(i)

    visited = [False for i in graph.keys()]
    components = []
    component_list = []
    for ind in graph.keys():
        if not visited[ind]:
            component_list = [ind]
            _visit_val(ind)
            components.append(component_list)
            print("Added {} to {}".format(component_list, components))

    return components

def flare_in_component(component_list, rows):
    # Returns a list of indices that are part of a component that had at least one node flare at least once in the 24hrs
    # following t_rec, as given by the dataframe rows in rows.
    flare_indices = []
    for component in component_list:
        has_flare = False
        for ind in component:
            if rows.loc[rows.index[ind],"any_flare_in_24h"] != 0:
                has_flare = True
        if has_flare:
            flare_indices = np.append(flare_indices, component)
    return flare_indices

def get_mean_dist(graph):
    sum = 0
    for key in graph.keys():
        distances = np.delete(graph[key],key)
        if len(distances) != 0:
            sum += np.sum(distances)/len(distances)
    return sum / len(graph.keys())


In [43]:
dists = calculate_distance_graph(vals)
# thresh = get_mean_dist(dists)
thresh = 40
mean_thresh = thresholded_distance_graph(dists, thresh)
print("Full graph: ",dists)
print("Thresholded at {}, got {}".format(thresh, mean_thresh))
components = get_connected_components(mean_thresh)
print(components)

Full graph:  {0: array([        inf, 48.42857926, 33.83161229, 66.13785027, 71.75790984,
       20.40540217]), 1: array([48.42857926,         inf, 69.67185634, 71.35165225, 67.1702811 ,
       60.45879757]), 2: array([ 33.83161229,  69.67185634,          inf,  40.52252319,
       104.57685366,  47.29032508]), 3: array([ 66.13785027,  71.35165225,  40.52252319,          inf,
       133.99564349,  84.78531272]), 4: array([ 71.75790984,  67.1702811 , 104.57685366, 133.99564349,
                inf,  57.96984133]), 5: array([20.40540217, 60.45879757, 47.29032508, 84.78531272, 57.96984133,
               inf])}
Thresholded at 40, got {0: array([        inf,  0.        , 33.83161229,  0.        ,  0.        ,
       20.40540217]), 1: array([ 0., inf,  0.,  0.,  0.,  0.]), 2: array([33.83161229,  0.        ,         inf,  0.        ,  0.        ,
        0.        ]), 3: array([ 0.,  0.,  0., inf,  0.,  0.]), 4: array([ 0.,  0.,  0.,  0., inf,  0.]), 5: array([20.40540217,  0.        ,  0.   

In [44]:
flare_in_component(components, vals)

Checking component  [0, 2, 5]
Checking index 0 in component
Checking index 2 in component
Checking index 5 in component
Component has_flare:  True
Checking component  [1]
Checking index 1 in component
Component has_flare:  False
Checking component  [3]
Checking index 3 in component
Component has_flare:  False
Checking component  [4]
Checking index 4 in component
Component has_flare:  False


array([0., 2., 5.])

In [28]:
print("HC X: {}-{}".format(data.loc[:,"hc_x"].max(), data.loc[:,"hc_x"].min()))
print("HC Y: {}-{}".format(data.loc[:,"hc_y"].max(), data.loc[:,"hc_y"].min()))

x_max = int(ceil(data.loc[:,"hc_x"].max()))
x_min = int(ceil(data.loc[:,"hc_x"].min()))
y_max = int(ceil(data.loc[:,"hc_y"].max()))
y_min = int(ceil(data.loc[:,"hc_y"].min()))



HC X: 693755.2286566767--694199.9995112926
HC Y: 496332.19956688327--447951.87804304756


In [52]:
step = 100

# x_list = np.zeros(int(max(x_max, abs(x_min))/step) * 2 + 1)
# y_list = np.zeros(int(max(y_max, abs(y_min))/step) * 2 +1)

x_size = int(max(x_max, abs(x_min))/step) * 2 + 1
y_size = int(max(y_max, abs(y_min))/step) * 2 + 1

data_list = np.zeros((x_size, y_size))

for x_val, y_val in zip(data.loc[:10, "hc_x"], data.loc[:10, "hc_y"]):
    xind = int(floor(x_val/step) + x_size/2)
    yind = int(floor(y_val/step) + y_size/2)
    data_list[xind, yind] += 1


ADVISOR TALK
Networks-style correlation analysis
Google spacial statistics - relationships of spacial relations
Bachelors thesis can be partial progress in a few different things
Good to be working on novel work
Ask Varad about astropy 
Schedule presentation****
Final Report D: 
Written - complete chunk of thesis, don't need to get all in one place
Presentation - brainstorming, fun, lighter weight 
*** Decide in next week - will you have a chunk of stuff done or still be working? BY MONDAY 
Think about doing both or neither - don't ditch work just to persue more interesting stuff 
She thinks google is evil lmaoooo
Good for grad school
See if their are actual papers using location***
People get mad when ML models work using data they don't think is important 

In [ ]:
plt.figure(figsize=[20,10])
plt.imshow(data_list, cmap='gray')